# memstack-agent 事件系统和状态管理

本 notebook 演示 memstack-agent 的事件系统和状态管理。

## 概述

memstack-agent 使用事件驱动架构：
- **EventType** - 30+ 种事件类型
- **EventCategory** - 事件分类（agent/hitl/sandbox/system/message）
- **ProcessorState** - 处理器状态机
- **AgentEvent** - 不可变事件基类

## 1. 导入必要模块

In [ ]:
import sys
sys.path.insert(0, '/Users/tiejunsun/github/agi-demos/src')

from memstack_agent import (
    EventType,
    AgentEvent,
    ThoughtEvent,
    ActEvent,
    ObserveEvent,
    TextStartEvent,
    TextDeltaEvent,
    TextEndEvent,
    ErrorEvent,
)
from memstack_agent.core.events import CompleteEvent  # 需要从 events 模块直接导入
from memstack_agent.core.types import (
    ProcessorState,
    EventCategory,
    get_event_category,
    is_terminal_event,
)
import json

## 2. 处理器状态机

处理器状态表示 Agent 在 ReAct 循环中的当前状态。

In [ ]:
# 查看所有处理器状态
print("Processor States:")
for state in ProcessorState:
    print(f"  - {state.value}: {state.name}")

In [ ]:
# 状态转换示例
state_transitions = [
    (ProcessorState.IDLE, ProcessorState.THINKING, "开始处理"),
    (ProcessorState.THINKING, ProcessorState.ACTING, "决定调用工具"),
    (ProcessorState.ACTING, ProcessorState.OBSERVING, "执行工具中"),
    (ProcessorState.OBSERVING, ProcessorState.THINKING, "处理工具结果"),
    (ProcessorState.THINKING, ProcessorState.COMPLETED, "生成最终答案"),
]

print("State Transitions:")
for from_state, to_state, description in state_transitions:
    print(f"  {from_state.value} -> {to_state.value}: {description}")

## 3. 事件类型和分类

In [ ]:
# 查看所有事件类型
print("Event Types:")
for i, et in enumerate(EventType):
    category = get_event_category(et)
    print(f"  {i+1:2d}. {et.value:30s} [{category.value}]")

In [ ]:
# 按类别分组事件
from collections import defaultdict

events_by_category = defaultdict(list)
for et in EventType:
    events_by_category[get_event_category(et).value].append(et.value)

print("Events by Category:")
for category, events in events_by_category.items():
    print(f"\n{category.upper()}:")
    for event in events:
        print(f"  - {event}")

## 4. 创建事件

所有事件都是不可变的（frozen dataclass）。

In [ ]:
# 创建思考事件
thought = ThoughtEvent(
    conversation_id="conv-123",
    content="我需要先搜索互联网获取最新信息...",
    step_index=1,
    thought_level="task"
)

print("Thought Event:")
print(json.dumps(thought.to_dict(), indent=2, ensure_ascii=False))

In [ ]:
# 创建工具调用事件
act = ActEvent(
    conversation_id="conv-123",
    tool_name="search_web",
    tool_input={"query": "Python async best practices"},
    call_id="call-001",
    tool_execution_id="exec-001"
)

print("Act Event:")
print(json.dumps(act.to_dict(), indent=2, ensure_ascii=False))

In [ ]:
# 创建观察事件
observe = ObserveEvent(
    conversation_id="conv-123",
    tool_name="search_web",
    result="Found 5 results about Python async...",
    duration_ms=350,
    call_id="call-001",
    tool_execution_id="exec-001"
)

print("Observe Event:")
print(json.dumps(observe.to_dict(), indent=2, ensure_ascii=False))

## 5. 流式文本事件

文本响应使用 text_start -> text_delta* -> text_end 序列。

In [ ]:
# 模拟流式文本生成
text_chunks = [
    "Python ",
    "异步编程 ",
    "的最佳实践 ",
    "包括使用 ",
    "asyncio 库。"
]

conversation_id = "conv-123"

# 开始事件
text_start = TextStartEvent(conversation_id=conversation_id)
print(f"1. {text_start.event_type.value}")

# 增量事件
full_text = ""
for i, chunk in enumerate(text_chunks):
    delta = TextDeltaEvent(conversation_id=conversation_id, delta=chunk)
    full_text += chunk
    print(f"2.{i+1}. {delta.event_type.value}: '{chunk}'")

# 结束事件
text_end = TextEndEvent(conversation_id=conversation_id, full_text=full_text)
print(f"3. {text_end.event_type.value}: '{full_text}'")

## 6. 终端事件

complete 和 error 是终端事件，表示流结束。

In [ ]:
# 检查是否是终端事件
print("Terminal events:")
for et in EventType:
    if is_terminal_event(et):
        print(f"  - {et.value}")

In [ ]:
# 创建完成事件
complete = CompleteEvent(
    conversation_id="conv-123",
    result="任务完成",
    tokens={"prompt": 500, "completion": 200, "total": 700},
    cost=0.015
)

print("Complete Event:")
print(json.dumps(complete.to_dict(), indent=2, ensure_ascii=False))

In [ ]:
# 创建错误事件
error = ErrorEvent(
    conversation_id="conv-123",
    message="API 请求超时",
    code="TIMEOUT",
    details={"timeout_seconds": 30}
)

print("Error Event:")
print(json.dumps(error.to_dict(), indent=2, ensure_ascii=False))

## 7. 事件的不可变性

所有事件都是 frozen dataclass，创建后无法修改。

In [ ]:
# 尝试修改事件会抛出异常
event = ThoughtEvent(
    conversation_id="conv-123",
    content="Original thought"
)

try:
    event.content = "Modified thought"
except Exception as e:
    print(f"Error (expected): {type(e).__name__}")
    print(f"Message: {e}")

## 8. 完整事件流示例

模拟一个完整的 Agent 执行流程。

In [ ]:
def simulate_agent_execution(conversation_id: str, user_message: str):
    """模拟 Agent 执行流程，生成事件序列。"""
    events = []
    
    # 1. 开始思考
    events.append(ThoughtEvent(
        conversation_id=conversation_id,
        content=f"用户询问: {user_message}。我需要先搜索相关信息。",
        step_index=1
    ))
    
    # 2. 调用搜索工具
    events.append(ActEvent(
        conversation_id=conversation_id,
        tool_name="search_web",
        tool_input={"query": user_message},
        tool_execution_id="exec-001"
    ))
    
    # 3. 获取搜索结果
    events.append(ObserveEvent(
        conversation_id=conversation_id,
        tool_name="search_web",
        result="搜索结果: Python asyncio 是 Python 的异步编程库...",
        duration_ms=250,
        tool_execution_id="exec-001"
    ))
    
    # 4. 生成回答
    events.append(TextStartEvent(conversation_id=conversation_id))
    
    answer = "根据搜索结果，Python asyncio 是用于异步编程的标准库。"
    for word in answer.split():
        events.append(TextDeltaEvent(conversation_id=conversation_id, delta=word + " "))
    
    events.append(TextEndEvent(conversation_id=conversation_id, full_text=answer))
    
    # 5. 完成
    events.append(CompleteEvent(
        conversation_id=conversation_id,
        tokens={"total": 500},
        cost=0.01
    ))
    
    return events


# 执行模拟
events = simulate_agent_execution("conv-123", "什么是 Python asyncio?")

print("Event Flow:")
print("=" * 60)
for i, event in enumerate(events):
    print(f"{i+1:2d}. [{event.event_type.value:20s}] {event.__class__.__name__}")

In [ ]:
# 完整事件流 JSON 输出
print("Full Event Flow (JSON):")
print("=" * 60)
for event in events:
    print(json.dumps(event.to_dict(), ensure_ascii=False))

## 总结

本 notebook 演示了 memstack-agent 的事件系统：

1. **ProcessorState** - 处理器状态机（11 种状态）
2. **EventType** - 30+ 种事件类型
3. **EventCategory** - 事件分类
4. **不可变事件** - 所有事件都是 frozen dataclass
5. **流式文本** - text_start -> text_delta* -> text_end
6. **终端事件** - complete/error 标志流结束

下一步：查看 `03-custom-tools.ipynb` 学习自定义工具实现。